In [5]:
import pandas as pd
import geopandas as gpd

from dotenv import load_dotenv
import os

In [7]:
load_dotenv(override=True)
# Ahora puedes acceder a las variables
ruta_unidad = os.getenv('RUTA_UNIDAD_ONE_DRIVE')
ruta_drive = os.getenv('RUTA_LOCAL_ONE_DRIVE')
print(ruta_unidad + ruta_drive)

G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON


In [8]:
path_grupos = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'
path_catastro = ruta_unidad + r'\OneDrive - Ingenio Azucarero Guabira S.A\INFORMACION GENERAL\CATASTRO\NUEVO\CATASTRO_S20_DELTAX.shp'
path_variedades = ruta_unidad + r"\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DELTA_X\_DETALLE_VARIEDAD.xlsx"

output_path = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DELTA_X'

In [9]:
def crear_xlsx_caneros():
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Seleccionar solo las columnas necesarias
    df_grupos = df_grupos[['CODIGO CAÑERO', 'NOMBRE CAÑERO', 'INSTITUCION']]
    # Renombrar las columnas
    df_grupos = df_grupos.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'NOMBRE CAÑERO': 'NOMBRE',
        'INSTITUCION': 'COD_INSTIT'
    })
    # Reordenar las columnas
    df_grupos = df_grupos[['COD_CANERO', 'COD_INSTIT', 'NOMBRE']]
    print(len(df_grupos), 'Cañeros registrados')
    archivo_salida = os.path.join(output_path, 'CANEROS.xlsx')
    df_grupos.to_excel(archivo_salida, index=False)
    print('Se exporto CANHEROS.xlsx exitosamente')

In [10]:
def crear_xlsx_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro.copy()
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para PROPIEDAD.xlsx')
    df_pivot = gdf_cat_filtrado.groupby(['unidad_01', 'unidad_02'])['area'].sum().reset_index()
    df_pivot['unidad_01'] = df_pivot['unidad_01'].astype(int)
    df_repetidos = df_pivot[df_pivot.duplicated(subset='unidad_01', keep=False)]
    print('Cantidad de repetidos:', len(df_repetidos))
    df_pivot = df_pivot.rename(columns={
        'unidad_01': 'COD_PROPIE',
        'unidad_02': 'NOMBRE_PRO',
        'area': 'AREA_TOTAL'
    })
    # Definir nombre de archivo
    archivo_salida = os.path.join(output_path, 'PROPIEDAD.xlsx')
    # Exportar a Excel
    df_pivot.to_excel(archivo_salida, index=False)
    print('Se exporto PROPIEDAD.xlsx exitosamente')
    return df_pivot

In [11]:
def crear_xlsx_caneho_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro.copy()
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacion CANERO-PROPIEDAD')
    # Asegurar que estamos trabajando sobre una copia, no una vista
    df_cat_filtrado = gdf_cat_filtrado[['unidad_03', 'unidad_01']].copy()
    # Convertir a entero
    df_cat_filtrado['unidad_03'] = df_cat_filtrado['unidad_03'].astype(int)
    df_cat_filtrado['unidad_01'] = df_cat_filtrado['unidad_01'].astype(int)
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Asegurarse de que ambas columnas estén en tipo entero
    df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
    # Realizar el merge (left join para mantener solo los registros del shapefile filtrado)
    df_combinado = df_cat_filtrado.merge(
        df_grupos,
        left_on='unidad_03',
        right_on='CODIGO CAÑERO',
        how='left'
    )
    # Eliminar los registros donde no se encontró coincidencia (por ejemplo, no tienen nombre cañero)
    df_combinado = df_combinado[df_combinado['NOMBRE CAÑERO'].notna()]
    # Seleccionar columnas específicas
    df_combinado = df_combinado[['unidad_01', 'GRUPO DE COSECHA']].copy()
    # Convertir las columnas a enteros
    df_combinado['unidad_01'] = df_combinado['unidad_01'].astype(int)
    df_combinado['GRUPO DE COSECHA'] = df_combinado['GRUPO DE COSECHA'].astype(int)
    # Crear columna 'idd' con el formato unidad_01_GRUPODECOSECHA
    df_combinado['idd'] = df_combinado['unidad_01'].astype(str) + '_' + df_combinado['GRUPO DE COSECHA'].astype(str)
    # Eliminar duplicados según la columna 'idd'
    df_combinado = df_combinado.drop_duplicates(subset='idd')
    # Crear una lista para guardar los resultados de cada grupo
    relaciones = []
    # Obtener todos los grupos únicos
    grupos = sorted(df_combinado['GRUPO DE COSECHA'].unique())
    for grupo in grupos:
        # Filtrar cañeros y propiedades del grupo actual
        df_cañeros = df_grupos[df_grupos['GRUPO DE COSECHA'] == grupo][['CODIGO CAÑERO']].drop_duplicates()
        df_props = df_combinado[df_combinado['GRUPO DE COSECHA'] == grupo][['unidad_01']].drop_duplicates()
        # Si hay cañeros y propiedades, hacer el producto cartesiano
        if not df_cañeros.empty and not df_props.empty:
            df_temp = df_cañeros.merge(df_props, how='cross')
            df_temp['GRUPO DE COSECHA'] = grupo
            relaciones.append(df_temp)
    # Unir todos los grupos en un solo DataFrame
    df_relacion_total = pd.concat(relaciones, ignore_index=True)
    # Eliminar la columna de grupo
    df_relacion_total = df_relacion_total.drop(columns=['GRUPO DE COSECHA'])
    # Renombrar las columnas
    df_relacion_total = df_relacion_total.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'unidad_01': 'COD_PROPIE'
    })
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'CANERO_PROPIEDAD.xlsx')
    # Exportar a Excel
    df_relacion_total.to_excel(archivo_salida, index=False)
    print('Se exporto CANERO_PROPIEDAD.xlsx exitosamente')
    return df_relacion_total

In [14]:
def crear_xlsx_lote_siembra_variedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro.copy()
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacaion LOTE-SIEMBRA-CANA')
    df_datos = gdf_cat_filtrado[['unidad_01', 'unidad_03', 'unidad_05', 'area', 'variedad']]
    df_datos_orden = df_datos.sort_values(by=['unidad_01', 'unidad_05'])
    df_datos_orden['COD_LOTE'] = (
        df_datos_orden.groupby('unidad_01').cumcount() + 1
    )
    df_datos_orden.head(60)
    df_variedades = pd.read_excel(path_variedades)
    # Merge con indicador para identificar las filas sin coincidencia
    df_mergeado = df_datos_orden.merge(
        df_variedades[['COD_CANA', 'VARIEDAD', 'CATASTRO']],
        left_on='variedad',
        right_on='CATASTRO',
        how='left',
        indicator=True # crea un campo _merge que indica si tiene relacion o no
    )
    df_sin_relacion = df_mergeado[df_mergeado['_merge'] == 'left_only']
    df_con_relacion = df_mergeado[df_mergeado['_merge'] == 'both']
    df_con_relacion = df_con_relacion.copy()
    df_con_relacion.drop(columns=['_merge'], inplace=True)
    df_con_relacion.rename(columns={
        'unidad_01': 'COD_PROPIE',
        'unidad_03': 'COD_CANERO',
        'unidad_05': 'NOMBRE_LOT',
        'area': 'AREA'}, inplace=True)
    df_con_relacion['COD_SIEMBR'] = range(1, len(df_con_relacion) + 1)
    lote = df_con_relacion[['COD_LOTE', 'COD_PROPIE', 'NOMBRE_LOT', 'AREA']]
    siembra = df_con_relacion[['COD_SIEMBR', 'COD_CANERO', 'COD_PROPIE', 'COD_LOTE', 'COD_CANA']]
    cana = df_con_relacion[['COD_CANA', 'VARIEDAD']].drop_duplicates(subset='COD_CANA')
    cana = cana.sort_values(by='COD_CANA')
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'LOTE.xlsx')
    # Exportar a Excel
    lote.to_excel(archivo_salida, index=False)
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'SIEMBRA.xlsx')
    # Exportar a Excel
    siembra.to_excel(archivo_salida, index=False)
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'CANA.xlsx')
    # Exportar a Excel
    cana.to_excel(archivo_salida, index=False)

    print('Area total procesada:', lote['AREA'].sum())
    print('Area con error variedad:', df_sin_relacion['area'].sum())
    print('Se creo LOTE.xlsx, SIEMBRA.xlsx, CANA.xlsx')

In [15]:
crear_xlsx_caneros()
prop_data = crear_xlsx_propiedad()
ca_prop_data = crear_xlsx_caneho_propiedad()
crear_xlsx_lote_siembra_variedad()

1417 Cañeros registrados
Se exporto CANHEROS.xlsx exitosamente
Area Total de Catastro: 54859.33609207103 ha. para PROPIEDAD.xlsx
Cantidad de repetidos: 0
Se exporto PROPIEDAD.xlsx exitosamente
Area Total de Catastro: 54859.33609207103 ha. para relacion CANERO-PROPIEDAD
Se exporto CANERO_PROPIEDAD.xlsx exitosamente
Area Total de Catastro: 54859.33609207103 ha. para relacaion LOTE-SIEMBRA-CANA
Area total procesada: 54841.310826612535
Area con error variedad: 18.0252654585
Se creo LOTE.xlsx, SIEMBRA.xlsx, CANA.xlsx


In [28]:
ca_prop_data.rename(columns={
    'COD_CANERO': 'unidad_03',
    'COD_PROPIE': 'unidad_01'
}, inplace=True)

In [29]:
# Agregar las columnas nuevas con los mismos valores
ca_prop_data['DESCRIPCIO'] = ''
ca_prop_data['CLASIFICAD'] = 'CO-IAG'
ca_prop_data['CONTA'] = 'B'

In [30]:
ca_prop_data

,unidad_03,unidad_01,DESCRIPCIO,CLASIFICAD,CONTA
0,515,17,,CO-IAG,B
1,515,1696,,CO-IAG,B
2,515,1748,,CO-IAG,B
3,1215,17,,CO-IAG,B
4,1215,1696,,CO-IAG,B
...,...,...,...,...,...
6098,266666,2278,,CO-IAG,B
6099,100000,2286,,CO-IAG,B
6100,100001,2281,,CO-IAG,B
6101,377777,1603,,CO-IAG,B


In [32]:
prop_data.rename(columns={
    'COD_PROPIE': 'unidad_01',
    'NOMBRE_PRO': 'unidad_02',
    'AREA_TOTAL': 'area'
}, inplace=True)

In [33]:
prop_data

,unidad_01,unidad_02,area
0,1,SONIMA,100.831841
1,2,JOSE TREJO--IRIARTE SEGUNDINA,0.801368
2,3,PROPIEDAD ONOFRE,22.617643
3,4,SAN JORGE--CATALA,35.135873
4,5,SAN FELIX--CATALA,201.292501
...,...,...,...
1667,2285,LA VICTORIA III--GONZALES LUIS,39.666443
1668,2286,PROPIEDAD MONTENEGRO,13.085574
1669,2287,LA TERAPIA,0.150080
1670,2288,CHACO LEJO--SALO SABINA,5.007837


In [34]:
gdf_catastro = gpd.read_file(path_catastro)
gdf_cat_filtrado = gdf_catastro[
    (gdf_catastro['obs'] == 'ok') &
    (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para PROPIEDAD.xlsx')

Area Total de Catastro: 56139.12697181613 ha. para PROPIEDAD.xlsx


In [35]:
invalidos = gdf_cat_filtrado[~gdf_cat_filtrado.is_valid]

In [36]:
invalidos[['fs']]

,fs
508,2023-06-27
10369,2022-05-11


In [37]:
import geopandas as gpd
import pandas as pd
import numpy as np

# Asegura geometrías válidas
gdf_cat_filtrado = gdf_cat_filtrado[gdf_cat_filtrado.is_valid]

# 🔄 Reproyectar a WGS84 (EPSG:4326)
gdf_cat_filtrado = gdf_cat_filtrado.to_crs(epsg=4326)

vertices_data = []

# Extraer vértices con atributos
for idx, row in gdf_cat_filtrado.iterrows():
    geometry = row.geometry
    if geometry.geom_type == 'Polygon':
        polys = [geometry]
    elif geometry.geom_type == 'MultiPolygon':
        polys = geometry.geoms
    else:
        continue

    for poly in polys:
        coords = list(poly.exterior.coords)
        for x, y in coords:
            data = row.drop('geometry').to_dict()
            data.update({'POINT_X': x, 'POINT_Y': y})
            vertices_data.append(data)

# Crear DataFrame
vertices_df = pd.DataFrame(vertices_data)

# Agregar columnas faltantes
vertices_df['OBJECTID'] = np.arange(1, len(vertices_df)+1)
vertices_df['Shape'] = 'Point ZM'
vertices_df['POINT_Z'] = 0
vertices_df['POINT_M'] = np.nan

# Convertir fs (fecha) a número Excel
vertices_df['fs'] = pd.to_datetime(vertices_df['fs'], errors='coerce')
excel_base = pd.Timestamp('1899-12-30')
vertices_df['fs'] = (vertices_df['fs'] - excel_base).dt.days.fillna(0).astype(int)

# Convertir columnas a enteros
cols_int = ['unidad_01', 'unidad_03', 'soca', 'zona']
vertices_df[cols_int] = vertices_df[cols_int].fillna(0).astype(int)

# Redondear coordenadas a 8 decimales
vertices_df['POINT_X'] = vertices_df['POINT_X'].round(8)
vertices_df['POINT_Y'] = vertices_df['POINT_Y'].round(8)

# Ordenar columnas como el Excel
columnas_finales = [
    'OBJECTID', 'Shape', 'id', 'unidad_01', 'unidad_02', 'unidad_03', 'unidad_04',
    'unidad_05', 'variedad', 'fs', 'textura', 'financia', 'soca', 'cultivo',
    'zona', 'area', 'POINT_X', 'POINT_Y', 'POINT_Z', 'POINT_M'
]

vertices_df = vertices_df[columnas_finales].sort_values(by='unidad_01').reset_index(drop=True)

In [38]:
vertices_df

,OBJECTID,Shape,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,fs,textura,financia,soca,cultivo,zona,area,POINT_X,POINT_Y,POINT_Z,POINT_M
0,107932,Point ZM,000010000L13.3,1,SONIMA,388,AGROPECUARIA MARIANA S.R.L.,L13.3,CITTCA0563,44659,F,IAG,2,canha,69,1.775559,-63.077887,-17.286836,0,NaN
1,107747,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083055,-17.288103,0,NaN
2,107746,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083647,-17.287868,0,NaN
3,107745,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083805,-17.287818,0,NaN
4,107744,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083836,-17.288163,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148826,148760,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,None,None,4,canha,304,48.264616,-62.862657,-17.262170,0,NaN
148827,148759,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,None,None,4,canha,304,48.264616,-62.862646,-17.262170,0,NaN
148828,148758,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,None,None,4,canha,304,48.264616,-62.862579,-17.262138,0,NaN
148829,148756,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,None,None,4,canha,304,48.264616,-62.862508,-17.261855,0,NaN


In [40]:
archivo_salida = os.path.join(output_path, 'CATASTRO_2025.xlsx')

In [42]:
# Crear un archivo Excel con múltiples hojas
with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
    vertices_df.to_excel(writer, sheet_name='CATASTRO', index=False)
    prop_data.to_excel(writer, sheet_name='PROPIEDADES', index=False)
    ca_prop_data.to_excel(writer, sheet_name='CODIGO_CAÑERO_PROPIEDAD', index=False)